In [1]:
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [2]:
!pip install fancyimpute


In [3]:
!pip install ace_tools


In [4]:
df = pd.read_csv("VN_housing_dataset.csv")
df

,Unnamed: 0,Ngày,Địa chỉ,Quận,Huyện,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2
0,0.0,2020-08-05,"Đường Hoàng Quốc Việt, Phường Nghĩa Đô, Quận C...",Quận Cầu Giấy,Phường Nghĩa Đô,"Nhà ngõ, hẻm",Đã có sổ,4,5 phòng,46 m²,NaN,NaN,"86,96 triệu/m²"
1,1.0,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà mặt phố, mặt tiền",NaN,NaN,3 phòng,37 m²,NaN,NaN,"116,22 triệu/m²"
2,2.0,2020-08-05,"phố minh khai, Phường Minh Khai, Quận Hai Bà T...",Quận Hai Bà Trưng,Phường Minh Khai,"Nhà ngõ, hẻm",Đã có sổ,4,4 phòng,40 m²,10 m,4 m,65 triệu/m²
3,3.0,2020-08-05,"Đường Võng Thị, Phường Thụy Khuê, Quận Tây Hồ,...",Quận Tây Hồ,Phường Thụy Khuê,"Nhà ngõ, hẻm",Đã có sổ,NaN,6 phòng,51 m²,12.75 m,4 m,100 triệu/m²
4,4.0,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà ngõ, hẻm",NaN,NaN,4 phòng,36 m²,9 m,4 m,"86,11 triệu/m²"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82492,82492.0,2019-08-07,"Đường Trần Quốc Hoàn, Phường Quan Hoa, Quận Cầ...",Quận Cầu Giấy,Phường Quan Hoa,"Nhà mặt phố, mặt tiền",NaN,NaN,3 phòng,50 m²,NaN,NaN,292 triệu/m²
82493,82493.0,2019-08-07,"Đường Nguyễn Khánh Toàn, Phường Quan Hoa, Quận...",Quận Cầu Giấy,Phường Quan Hoa,"Nhà mặt phố, mặt tiền",Đã có sổ,NaN,4 phòng,41 m²,NaN,NaN,"341,46 triệu/m²"
82494,82494.0,2019-08-05,"Đường Quan Hoa, Phường Quan Hoa, Quận Cầu Giấy...",Quận Cầu Giấy,Phường Quan Hoa,"Nhà ngõ, hẻm",Đã có sổ,NaN,4 phòng,60 m²,NaN,NaN,"101,67 triệu/m²"
82495,82495.0,2019-08-05,"Đường Hồ Tùng Mậu, Phường Mai Dịch, Quận Cầu G...",Quận Cầu Giấy,Phường Mai Dịch,Nhà phố liền kề,NaN,NaN,4 phòng,45 m²,NaN,NaN,"102,22 triệu/m²"


In [5]:
# Kiểm tra và hiển thị số lượng giá trị thiếu của mỗi cột
missing_values = df.isnull().sum()
missing_values


,0
Unnamed: 0,1
Ngày,1
Địa chỉ,48
Quận,2
Huyện,48
Loại hình nhà ở,32
Giấy tờ pháp lý,28887
Số tầng,46098
Số phòng ngủ,39
Diện tích,2


In [6]:
df = df.dropna(subset=['Địa chỉ'])
df = df.dropna(subset=['Loại hình nhà ở'])


In [7]:
df.drop(columns=['Unnamed: 0'], inplace=True)
df.drop(columns=['Quận'], inplace=True)
df.drop(columns=['Huyện'], inplace=True)
df.head()

,Ngày,Địa chỉ,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2
0,2020-08-05,"Đường Hoàng Quốc Việt, Phường Nghĩa Đô, Quận C...","Nhà ngõ, hẻm",Đã có sổ,4,5 phòng,46 m²,NaN,NaN,"86,96 triệu/m²"
1,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...","Nhà mặt phố, mặt tiền",NaN,NaN,3 phòng,37 m²,NaN,NaN,"116,22 triệu/m²"
2,2020-08-05,"phố minh khai, Phường Minh Khai, Quận Hai Bà T...","Nhà ngõ, hẻm",Đã có sổ,4,4 phòng,40 m²,10 m,4 m,65 triệu/m²
3,2020-08-05,"Đường Võng Thị, Phường Thụy Khuê, Quận Tây Hồ,...","Nhà ngõ, hẻm",Đã có sổ,NaN,6 phòng,51 m²,12.75 m,4 m,100 triệu/m²
4,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...","Nhà ngõ, hẻm",NaN,NaN,4 phòng,36 m²,9 m,4 m,"86,11 triệu/m²"


In [8]:
missing_values_after_drop = df.isnull().sum()
missing_values_after_drop


,0
Ngày,0
Địa chỉ,0
Loại hình nhà ở,0
Giấy tờ pháp lý,28822
Số tầng,46020
Số phòng ngủ,28
Diện tích,1
Dài,62594
Rộng,46977
Giá/m2,12


In [9]:
def extract_number(text):
  if pd.isna(text):
      return None
  # Loại bỏ tất cả ký tự không phải số, dấu chấm hoặc dấu phẩy
  text = re.sub(r'[^\d.,]', '', text)
  # Thay dấu phẩy bằng dấu chấm để chuyển sang float
  text = text.replace(',', '.')
  try:
    return float(text)
  except:
    return None

In [10]:
# Xử lý từng cột -> giá trị số
df['Số phòng ngủ'] = df['Số phòng ngủ'].apply(lambda x: int(re.sub(r'\D', '', x)) if pd.notna(x) else None)
df['Diện tích'] = df['Diện tích'].apply(extract_number)
df['Dài'] = df['Dài'].apply(extract_number)
df['Rộng'] = df['Rộng'].apply(extract_number)
df['Giá/m2'] = df['Giá/m2'].apply(extract_number)

df.head()

,Ngày,Địa chỉ,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2
0,2020-08-05,"Đường Hoàng Quốc Việt, Phường Nghĩa Đô, Quận C...","Nhà ngõ, hẻm",Đã có sổ,4,5.0,46.0,NaN,NaN,86.96
1,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...","Nhà mặt phố, mặt tiền",NaN,NaN,3.0,37.0,NaN,NaN,116.22
2,2020-08-05,"phố minh khai, Phường Minh Khai, Quận Hai Bà T...","Nhà ngõ, hẻm",Đã có sổ,4,4.0,40.0,10.00,4.0,65.00
3,2020-08-05,"Đường Võng Thị, Phường Thụy Khuê, Quận Tây Hồ,...","Nhà ngõ, hẻm",Đã có sổ,NaN,6.0,51.0,12.75,4.0,100.00
4,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...","Nhà ngõ, hẻm",NaN,NaN,4.0,36.0,9.00,4.0,86.11


**THỰC HIỆN TÁCH CỘT ĐỊA CHỈ**

In [11]:
# Hàm kiểm tra xem một địa chỉ có đúng 4 phần khi tách bằng dấu phẩy và có đúng định dạng không
def is_valid_address(address):
  parts = [p.strip() for p in address.split(',')]
  if len(parts) != 4:
    return False
  pattern_duong = r'^Đường\s+.+'
  pattern_phuong = r'^Phường\s+.+'
  pattern_quan = r'^Quận\s+.+'
  pattern_thanhpho = r'^Hà Nội$'
  return (re.match(pattern_duong, parts[0]) is not None and
          re.match(pattern_phuong, parts[1]) is not None and
          re.match(pattern_quan, parts[2]) is not None and
          re.match(pattern_thanhpho, parts[3]) is not None)

In [12]:
# Lọc ra các dòng ngoại lệ
df['valid_address'] = df['Địa chỉ'].apply(is_valid_address) #Tạo cột mới valid_address để đánh dấu cột địa chỉ có như cấu trúc không

df_invalid = df[~df['valid_address']]

# Hiển thị các ngoại lệ
print("Các dòng ngoại lệ:")
print(df_invalid['Địa chỉ'])

Các dòng ngoại lệ:
2        phố minh khai, Phường Minh Khai, Quận Hai Bà T...
10       Đường Tựu Liệt, Thị trấn Văn Điển, Huyện Thanh...
29       180/61/5, Đường Tây Mỗ, Phường Tây Mỗ, Quận Na...
34       Đường Tả Thanh Oai, Xã Tả Thanh Oai, Huyện Tha...
37       Đường Vĩnh Quỳnh, Xã Tam Hiệp, Huyện Thanh Trì...
                               ...                        
82397    Tam Khương, Phường Khương Thượng, Quận Đống Đa...
82401    Đường Lê Trọng Tấn Hoài Đức, Hà Nội, Xã Vân Ca...
82422    Đường Lạc Long Quân Tây Hồ, Hà Nội, Phường Phú...
82436    Số 7 Đường Đại lộ Thăng Long Nam Từ Liêm, Hà N...
82437    Phố lò đúc, Phường Thanh Lương, Quận Hai Bà Tr...
Name: Địa chỉ, Length: 14083, dtype: object


In [13]:
# Xóa các dòng ngoại lệ khỏi DataFrame chính
df_clean = df[df['valid_address']].copy()

# Xóa cột tạm 'valid_address'
df_clean.drop(columns=['valid_address'], inplace=True)

In [14]:
df_clean.head()

,Ngày,Địa chỉ,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2
0,2020-08-05,"Đường Hoàng Quốc Việt, Phường Nghĩa Đô, Quận C...","Nhà ngõ, hẻm",Đã có sổ,4,5.0,46.0,NaN,NaN,86.96
1,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...","Nhà mặt phố, mặt tiền",NaN,NaN,3.0,37.0,NaN,NaN,116.22
3,2020-08-05,"Đường Võng Thị, Phường Thụy Khuê, Quận Tây Hồ,...","Nhà ngõ, hẻm",Đã có sổ,NaN,6.0,51.0,12.75,4.0,100.00
4,2020-08-05,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...","Nhà ngõ, hẻm",NaN,NaN,4.0,36.0,9.00,4.0,86.11
5,2020-08-05,"Đường Yên Hòa, Phường Yên Hoà, Quận Cầu Giấy, ...","Nhà ngõ, hẻm",Đã có sổ,NaN,10.0,46.0,12.10,3.8,104.35


In [15]:
df_clean['Địa chỉ'] = df_clean['Địa chỉ'].str.replace(r',\s*Hà Nội$', '', regex=True)

print(df_clean['Địa chỉ'].iloc[0])


Đường Hoàng Quốc Việt, Phường Nghĩa Đô, Quận Cầu Giấy


In [16]:
df_split = df_clean['Địa chỉ'].str.split(',', expand=True)
df_split.columns = ['Đường', 'Phường', 'Quận']

for col in df_split.columns:
  df_split[col] = df_split[col].str.strip()

df_clean = pd.concat([df_clean, df_split], axis=1)

df_clean.drop(columns=['Địa chỉ'], inplace=True)

df_clean.head()

,Ngày,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2,Đường,Phường,Quận
0,2020-08-05,"Nhà ngõ, hẻm",Đã có sổ,4,5.0,46.0,NaN,NaN,86.96,Đường Hoàng Quốc Việt,Phường Nghĩa Đô,Quận Cầu Giấy
1,2020-08-05,"Nhà mặt phố, mặt tiền",NaN,NaN,3.0,37.0,NaN,NaN,116.22,Đường Kim Giang,Phường Kim Giang,Quận Thanh Xuân
3,2020-08-05,"Nhà ngõ, hẻm",Đã có sổ,NaN,6.0,51.0,12.75,4.0,100.00,Đường Võng Thị,Phường Thụy Khuê,Quận Tây Hồ
4,2020-08-05,"Nhà ngõ, hẻm",NaN,NaN,4.0,36.0,9.00,4.0,86.11,Đường Kim Giang,Phường Kim Giang,Quận Thanh Xuân
5,2020-08-05,"Nhà ngõ, hẻm",Đã có sổ,NaN,10.0,46.0,12.10,3.8,104.35,Đường Yên Hòa,Phường Yên Hoà,Quận Cầu Giấy


**THỰC HIỆN TÁCH CỘT NGÀY**

In [17]:
df_clean['Ngày'] = pd.to_datetime(df_clean['Ngày'])

In [18]:
df_clean['Năm'] = df_clean['Ngày'].dt.year
df_clean['Tháng'] = df_clean['Ngày'].dt.month
df_clean['Ngày_'] = df_clean['Ngày'].dt.day


In [19]:
df_clean.drop(columns=['Ngày'], inplace=True)

df_clean.head()

,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2,Đường,Phường,Quận,Năm,Tháng,Ngày_
0,"Nhà ngõ, hẻm",Đã có sổ,4,5.0,46.0,NaN,NaN,86.96,Đường Hoàng Quốc Việt,Phường Nghĩa Đô,Quận Cầu Giấy,2020,8,5
1,"Nhà mặt phố, mặt tiền",NaN,NaN,3.0,37.0,NaN,NaN,116.22,Đường Kim Giang,Phường Kim Giang,Quận Thanh Xuân,2020,8,5
3,"Nhà ngõ, hẻm",Đã có sổ,NaN,6.0,51.0,12.75,4.0,100.00,Đường Võng Thị,Phường Thụy Khuê,Quận Tây Hồ,2020,8,5
4,"Nhà ngõ, hẻm",NaN,NaN,4.0,36.0,9.00,4.0,86.11,Đường Kim Giang,Phường Kim Giang,Quận Thanh Xuân,2020,8,5
5,"Nhà ngõ, hẻm",Đã có sổ,NaN,10.0,46.0,12.10,3.8,104.35,Đường Yên Hòa,Phường Yên Hoà,Quận Cầu Giấy,2020,8,5


In [20]:

missing_values_ = df_clean.isnull().sum()
missing_values_


,0
Loại hình nhà ở,0
Giấy tờ pháp lý,24514
Số tầng,38429
Số phòng ngủ,20
Diện tích,1
Dài,53905
Rộng,40495
Giá/m2,10
Đường,0
Phường,0


**XỬ LÝ GIÁ TRỊ THIẾU CHO CÁC CỘT FLOAT**

In [21]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [22]:
df_clean

,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2,Đường,Phường,Quận,Năm,Tháng,Ngày_
0,"Nhà ngõ, hẻm",Đã có sổ,4,5.0,46.0,NaN,NaN,86.96,Đường Hoàng Quốc Việt,Phường Nghĩa Đô,Quận Cầu Giấy,2020,8,5
1,"Nhà mặt phố, mặt tiền",NaN,NaN,3.0,37.0,NaN,NaN,116.22,Đường Kim Giang,Phường Kim Giang,Quận Thanh Xuân,2020,8,5
3,"Nhà ngõ, hẻm",Đã có sổ,NaN,6.0,51.0,12.75,4.0,100.00,Đường Võng Thị,Phường Thụy Khuê,Quận Tây Hồ,2020,8,5
4,"Nhà ngõ, hẻm",NaN,NaN,4.0,36.0,9.00,4.0,86.11,Đường Kim Giang,Phường Kim Giang,Quận Thanh Xuân,2020,8,5
5,"Nhà ngõ, hẻm",Đã có sổ,NaN,10.0,46.0,12.10,3.8,104.35,Đường Yên Hòa,Phường Yên Hoà,Quận Cầu Giấy,2020,8,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82491,Nhà phố liền kề,NaN,NaN,3.0,38.0,NaN,NaN,81.58,Đường Hồ Tùng Mậu,Phường Phúc Diễn,Quận Bắc Từ Liêm,2019,8,23
82492,"Nhà mặt phố, mặt tiền",NaN,NaN,3.0,50.0,NaN,NaN,292.00,Đường Trần Quốc Hoàn,Phường Quan Hoa,Quận Cầu Giấy,2019,8,7
82493,"Nhà mặt phố, mặt tiền",Đã có sổ,NaN,4.0,41.0,NaN,NaN,341.46,Đường Nguyễn Khánh Toàn,Phường Quan Hoa,Quận Cầu Giấy,2019,8,7
82494,"Nhà ngõ, hẻm",Đã có sổ,NaN,4.0,60.0,NaN,NaN,101.67,Đường Quan Hoa,Phường Quan Hoa,Quận Cầu Giấy,2019,8,5


In [23]:
so_tang = df_clean['Số tầng'].value_counts()
print(so_tang)

Số tầng
5               13272
4               10042
3                2794
6                1793
2                 749
7                 513
1                 447
8                 163
9                  77
10                 30
Nhiều hơn 10        6
45                  3
35                  2
33                  1
42                  1
25                  1
73                  1
65                  1
55                  1
38                  1
30                  1
14                  1
40                  1
52                  1
54                  1
32                  1
12                  1
29                  1
Name: count, dtype: int64


In [24]:
df_clean['Số tầng'] = df_clean['Số tầng'].replace('Nhiều hơn 10', 11)
print(df_clean['Số tầng'].value_counts())


Số tầng
5     13272
4     10042
3      2794
6      1793
2       749
7       513
1       447
8       163
9        77
10       30
11        6
45        3
35        2
33        1
42        1
25        1
73        1
65        1
55        1
38        1
30        1
14        1
40        1
52        1
54        1
32        1
12        1
29        1
Name: count, dtype: int64


In [25]:
# Các cột cần điền gtri thiếu
cols_to_impute = ['Số tầng', 'Số phòng ngủ', 'Diện tích', 'Dài', 'Rộng', 'Giá/m2']

df_impute = df_clean.copy()

# chỉ chọn các cột cần điền
data_impute = df_impute[cols_to_impute]

# Khởi tạo
imputer = IterativeImputer(random_state=42)

imputed_array = imputer.fit_transform(data_impute)

# Gán lại dữ liệu đã điền cho DataFrame
df_impute[cols_to_impute] = imputed_array

In [26]:
print(df_impute[cols_to_impute].isnull().sum())
print(df_impute[cols_to_impute].head())

Số tầng         0
Số phòng ngủ    0
Diện tích       0
Dài             0
Rộng            0
Giá/m2          0
dtype: int64
    Số tầng  Số phòng ngủ  Diện tích        Dài       Rộng  Giá/m2
0  4.000000           5.0       46.0  95.639198  39.656084   86.96
1  4.182001           3.0       37.0  94.627033  40.220172  116.22
3  5.246178           6.0       51.0  12.750000   4.000000  100.00
4  4.475657           4.0       36.0   9.000000   4.000000   86.11
5  6.731277          10.0       46.0  12.100000   3.800000  104.35


In [27]:
df_impute['Số tầng'] = df_impute['Số tầng'].round().astype(int)

In [28]:
df_impute.head()

,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2,Đường,Phường,Quận,Năm,Tháng,Ngày_
0,"Nhà ngõ, hẻm",Đã có sổ,4,5.0,46.0,95.639198,39.656084,86.96,Đường Hoàng Quốc Việt,Phường Nghĩa Đô,Quận Cầu Giấy,2020,8,5
1,"Nhà mặt phố, mặt tiền",NaN,4,3.0,37.0,94.627033,40.220172,116.22,Đường Kim Giang,Phường Kim Giang,Quận Thanh Xuân,2020,8,5
3,"Nhà ngõ, hẻm",Đã có sổ,5,6.0,51.0,12.750000,4.000000,100.00,Đường Võng Thị,Phường Thụy Khuê,Quận Tây Hồ,2020,8,5
4,"Nhà ngõ, hẻm",NaN,4,4.0,36.0,9.000000,4.000000,86.11,Đường Kim Giang,Phường Kim Giang,Quận Thanh Xuân,2020,8,5
5,"Nhà ngõ, hẻm",Đã có sổ,7,10.0,46.0,12.100000,3.800000,104.35,Đường Yên Hòa,Phường Yên Hoà,Quận Cầu Giấy,2020,8,5


In [29]:
df_impute

,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2,Đường,Phường,Quận,Năm,Tháng,Ngày_
0,"Nhà ngõ, hẻm",Đã có sổ,4,5.0,46.0,95.639198,39.656084,86.96,Đường Hoàng Quốc Việt,Phường Nghĩa Đô,Quận Cầu Giấy,2020,8,5
1,"Nhà mặt phố, mặt tiền",NaN,4,3.0,37.0,94.627033,40.220172,116.22,Đường Kim Giang,Phường Kim Giang,Quận Thanh Xuân,2020,8,5
3,"Nhà ngõ, hẻm",Đã có sổ,5,6.0,51.0,12.750000,4.000000,100.00,Đường Võng Thị,Phường Thụy Khuê,Quận Tây Hồ,2020,8,5
4,"Nhà ngõ, hẻm",NaN,4,4.0,36.0,9.000000,4.000000,86.11,Đường Kim Giang,Phường Kim Giang,Quận Thanh Xuân,2020,8,5
5,"Nhà ngõ, hẻm",Đã có sổ,7,10.0,46.0,12.100000,3.800000,104.35,Đường Yên Hòa,Phường Yên Hoà,Quận Cầu Giấy,2020,8,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82491,Nhà phố liền kề,NaN,4,3.0,38.0,96.525675,39.721034,81.58,Đường Hồ Tùng Mậu,Phường Phúc Diễn,Quận Bắc Từ Liêm,2019,8,23
82492,"Nhà mặt phố, mặt tiền",NaN,5,3.0,50.0,84.811029,42.670071,292.00,Đường Trần Quốc Hoàn,Phường Quan Hoa,Quận Cầu Giấy,2019,8,7
82493,"Nhà mặt phố, mặt tiền",Đã có sổ,5,4.0,41.0,82.155131,43.451135,341.46,Đường Nguyễn Khánh Toàn,Phường Quan Hoa,Quận Cầu Giấy,2019,8,7
82494,"Nhà ngõ, hẻm",Đã có sổ,5,4.0,60.0,95.176981,39.940487,101.67,Đường Quan Hoa,Phường Quan Hoa,Quận Cầu Giấy,2019,8,5


XỬ LÝ GIÁ TRỊ THIẾU CỦA "GIẤY TỜ PHÁP LÝ" BẰNG MODE(GIÁ TRỊ XUẤT HIỆN NHIỀU NHẤT)

In [30]:

mode_value = df_impute['Giấy tờ pháp lý'].mode()[0]

# Điền giá trị thiếu bằng mode
df_impute['Giấy tờ pháp lý'] = df_impute['Giấy tờ pháp lý'].fillna(mode_value)

# Kiểm tra i
print(df_impute['Giấy tờ pháp lý'].isnull().sum())
print(df_impute['Giấy tờ pháp lý'].value_counts())


0
Giấy tờ pháp lý
Đã có sổ        67872
Giấy tờ khác      232
Đang chờ sổ       231
Name: count, dtype: int64


In [31]:
df_impute.head(20)

,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2,Đường,Phường,Quận,Năm,Tháng,Ngày_
0,"Nhà ngõ, hẻm",Đã có sổ,4,5.0,46.0,95.639198,39.656084,86.96,Đường Hoàng Quốc Việt,Phường Nghĩa Đô,Quận Cầu Giấy,2020,8,5
1,"Nhà mặt phố, mặt tiền",Đã có sổ,4,3.0,37.0,94.627033,40.220172,116.22,Đường Kim Giang,Phường Kim Giang,Quận Thanh Xuân,2020,8,5
3,"Nhà ngõ, hẻm",Đã có sổ,5,6.0,51.0,12.750000,4.000000,100.00,Đường Võng Thị,Phường Thụy Khuê,Quận Tây Hồ,2020,8,5
4,"Nhà ngõ, hẻm",Đã có sổ,4,4.0,36.0,9.000000,4.000000,86.11,Đường Kim Giang,Phường Kim Giang,Quận Thanh Xuân,2020,8,5
5,"Nhà ngõ, hẻm",Đã có sổ,7,10.0,46.0,12.100000,3.800000,104.35,Đường Yên Hòa,Phường Yên Hoà,Quận Cầu Giấy,2020,8,5
6,"Nhà ngõ, hẻm",Đã có sổ,4,3.0,52.0,14.524888,4.500000,112.50,Đường Tây Sơn,Phường Trung Liệt,Quận Đống Đa,2020,8,5
7,"Nhà mặt phố, mặt tiền",Đã có sổ,6,5.0,32.0,13.319576,6.800000,184.38,Đường Lò Đúc,Phường Đống Mác,Quận Hai Bà Trưng,2020,8,5
8,"Nhà ngõ, hẻm",Đã có sổ,5,4.0,75.0,12.000000,6.500000,120.00,Đường Xuân La,Phường Xuân La,Quận Tây Hồ,2020,8,5
9,"Nhà ngõ, hẻm",Đã có sổ,4,3.0,41.0,16.441247,3.500000,64.63,Đường 19/5,Phường Văn Quán,Quận Hà Đông,2020,8,5
11,"Nhà ngõ, hẻm",Đã có sổ,5,4.0,30.0,96.780540,39.923873,83.33,Đường Định Công Hạ,Phường Định Công,Quận Hoàng Mai,2020,8,5


**BẮT ĐẦU BƯỚC ONE-HOT ENCODING CHO DỮ LIỆU CATEGORICAL**

In [32]:
loai_hinh_nha = df_impute['Loại hình nhà ở'].value_counts()
print(loai_hinh_nha)

Loại hình nhà ở
Nhà ngõ, hẻm             52261
Nhà mặt phố, mặt tiền    14087
Nhà phố liền kề           1349
Nhà biệt thự               638
Name: count, dtype: int64


In [33]:
mapping_loai_hinh_nha = {
  "Nhà ngõ, hẻm": 1,
  "Nhà mặt phố, mặt tiền": 2,
  "Nhà phố liền kề": 3,
  "Nhà biệt thự": 4
}

# Áp dụng mapping vào cột "Loại hình nhà ở"
df_impute['Loại hình nhà ở_encode'] = df_impute['Loại hình nhà ở'].map(mapping_loai_hinh_nha)

# Sử dụng pd.get_dummies()
df_loai_hinh_nha = pd.get_dummies(df_impute['Loại hình nhà ở_encode'], prefix='Loai_hinh_nha')

# Gắn cột one-hot encoding vào dataframe gốc
df_impute = pd.concat([df_impute, df_loai_hinh_nha], axis=1)

df_impute.head()

,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2,Đường,Phường,Quận,Năm,Tháng,Ngày_,Loại hình nhà ở_encode,Loai_hinh_nha_1,Loai_hinh_nha_2,Loai_hinh_nha_3,Loai_hinh_nha_4
0,"Nhà ngõ, hẻm",Đã có sổ,4,5.0,46.0,95.639198,39.656084,86.96,Đường Hoàng Quốc Việt,Phường Nghĩa Đô,Quận Cầu Giấy,2020,8,5,1,True,False,False,False
1,"Nhà mặt phố, mặt tiền",Đã có sổ,4,3.0,37.0,94.627033,40.220172,116.22,Đường Kim Giang,Phường Kim Giang,Quận Thanh Xuân,2020,8,5,2,False,True,False,False
3,"Nhà ngõ, hẻm",Đã có sổ,5,6.0,51.0,12.750000,4.000000,100.00,Đường Võng Thị,Phường Thụy Khuê,Quận Tây Hồ,2020,8,5,1,True,False,False,False
4,"Nhà ngõ, hẻm",Đã có sổ,4,4.0,36.0,9.000000,4.000000,86.11,Đường Kim Giang,Phường Kim Giang,Quận Thanh Xuân,2020,8,5,1,True,False,False,False
5,"Nhà ngõ, hẻm",Đã có sổ,7,10.0,46.0,12.100000,3.800000,104.35,Đường Yên Hòa,Phường Yên Hoà,Quận Cầu Giấy,2020,8,5,1,True,False,False,False


In [34]:
giay_to = df_impute['Giấy tờ pháp lý'].value_counts()
print(giay_to)

Giấy tờ pháp lý
Đã có sổ        67872
Giấy tờ khác      232
Đang chờ sổ       231
Name: count, dtype: int64


In [35]:
mapping_giay_to = {
  "Đã có sổ": 1,
  "Giấy tờ khác": 2,
  "Đang chờ sổ": 3
}

# Áp dụng mapping vào cột "Giấy tờ pháp lý"
df_impute['Giấy tờ pháp lý_encode'] = df_impute['Giấy tờ pháp lý'].map(mapping_giay_to)

# Sử dụng pd.get_dummies()
df_giay_to = pd.get_dummies(df_impute['Giấy tờ pháp lý_encode'], prefix='Giay_to')

# Gắn cột one-hot encoding vào dataframe gốc
df_impute = pd.concat([df_impute, df_giay_to], axis=1)

df_impute.head()

,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2,Đường,Phường,...,Ngày_,Loại hình nhà ở_encode,Loai_hinh_nha_1,Loai_hinh_nha_2,Loai_hinh_nha_3,Loai_hinh_nha_4,Giấy tờ pháp lý_encode,Giay_to_1,Giay_to_2,Giay_to_3
0,"Nhà ngõ, hẻm",Đã có sổ,4,5.0,46.0,95.639198,39.656084,86.96,Đường Hoàng Quốc Việt,Phường Nghĩa Đô,...,5,1,True,False,False,False,1,True,False,False
1,"Nhà mặt phố, mặt tiền",Đã có sổ,4,3.0,37.0,94.627033,40.220172,116.22,Đường Kim Giang,Phường Kim Giang,...,5,2,False,True,False,False,1,True,False,False
3,"Nhà ngõ, hẻm",Đã có sổ,5,6.0,51.0,12.750000,4.000000,100.00,Đường Võng Thị,Phường Thụy Khuê,...,5,1,True,False,False,False,1,True,False,False
4,"Nhà ngõ, hẻm",Đã có sổ,4,4.0,36.0,9.000000,4.000000,86.11,Đường Kim Giang,Phường Kim Giang,...,5,1,True,False,False,False,1,True,False,False
5,"Nhà ngõ, hẻm",Đã có sổ,7,10.0,46.0,12.100000,3.800000,104.35,Đường Yên Hòa,Phường Yên Hoà,...,5,1,True,False,False,False,1,True,False,False


In [36]:
df_impute.drop(columns=['Loại hình nhà ở'], inplace=True)
df_impute.drop(columns=['Giấy tờ pháp lý'], inplace=True)
df_impute.head()

,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2,Đường,Phường,Quận,Năm,...,Ngày_,Loại hình nhà ở_encode,Loai_hinh_nha_1,Loai_hinh_nha_2,Loai_hinh_nha_3,Loai_hinh_nha_4,Giấy tờ pháp lý_encode,Giay_to_1,Giay_to_2,Giay_to_3
0,4,5.0,46.0,95.639198,39.656084,86.96,Đường Hoàng Quốc Việt,Phường Nghĩa Đô,Quận Cầu Giấy,2020,...,5,1,True,False,False,False,1,True,False,False
1,4,3.0,37.0,94.627033,40.220172,116.22,Đường Kim Giang,Phường Kim Giang,Quận Thanh Xuân,2020,...,5,2,False,True,False,False,1,True,False,False
3,5,6.0,51.0,12.750000,4.000000,100.00,Đường Võng Thị,Phường Thụy Khuê,Quận Tây Hồ,2020,...,5,1,True,False,False,False,1,True,False,False
4,4,4.0,36.0,9.000000,4.000000,86.11,Đường Kim Giang,Phường Kim Giang,Quận Thanh Xuân,2020,...,5,1,True,False,False,False,1,True,False,False
5,7,10.0,46.0,12.100000,3.800000,104.35,Đường Yên Hòa,Phường Yên Hoà,Quận Cầu Giấy,2020,...,5,1,True,False,False,False,1,True,False,False


In [37]:
duong = df_impute['Đường'].value_counts()
print(duong)

Đường
Đường Minh Khai              1641
Đường Khương Trung           1201
Đường Trương Định            1166
Đường Kim Giang              1108
Đường Lê Trọng Tấn           1094
                             ... 
Đường Cống Đá                   1
Đường Nguyễn Trường Tộ       1
Đường Mễ Trì Thượng||2284       1
Đường phú mỹ                    1
Đường 918 Quân Đội              1
Name: count, Length: 1457, dtype: int64


In [38]:
phuong = df_impute['Phường'].value_counts()
print(phuong)

Phường
Phường Khương Trung    2523
Phường Khương Đình     2316
Phường Minh Khai       1793
Phường Nhân Chính      1647
Phường Khương Mai      1443
                       ... 
Phường Đức Thắng          5
Phường Đồng Xuân          4
Phường Liên Mạc           4
Phường Tây Tựu            3
Phường Thượng Cát         2
Name: count, Length: 166, dtype: int64


In [39]:
quan = df_impute['Quận'].value_counts()
print(quan)

Quận
Quận Đống Đa         12110
Quận Thanh Xuân      11333
Quận Hoàng Mai        9582
Quận Hai Bà Trưng     9161
Quận Hà Đông          6162
Quận Cầu Giấy         5698
Quận Ba Đình          4077
Quận Long Biên        3302
Quận Nam Từ Liêm      2706
Quận Tây Hồ           2528
Quận Bắc Từ Liêm      1237
Quận Hoàn Kiếm         439
Name: count, dtype: int64


**Cột "Quận" áp dụng one-hot encoding, 2 cột "Dường" và "Phường" áp dụng Frequency Encoding **

In [40]:
mapping_quan = {
  "Quận Đống Đa": 1,
  "Quận Thanh Xuân": 2,
  "Quận Hoàng Mai": 3,
  "Quận Hai Bà Trưng": 4,
  "Quận Hà Đông": 5,
  "Quận Cầu Giấy": 6,
  "Quận Ba Đình": 7,
  "Quận Long Biên": 8,
  "Quận Nam Từ Liêm": 9,
  "Quận Tây Hồ": 10,
  "Quận Bắc Từ Liêm": 11,
  "Quận Hoàn Kiếm": 12
}

# Áp dụng mapping vào cột "Quận"
df_impute['Quận_encode'] = df_impute['Quận'].map(mapping_quan)

# Sử dụng pd.get_dummies()
df_quan = pd.get_dummies(df_impute['Quận_encode'], prefix='Quan_')

# Gắn cột one-hot encoding vào dataframe gốc
df_impute = pd.concat([df_impute, df_quan], axis=1)

df_impute.head()

,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2,Đường,Phường,Quận,Năm,...,Quan__3,Quan__4,Quan__5,Quan__6,Quan__7,Quan__8,Quan__9,Quan__10,Quan__11,Quan__12
0,4,5.0,46.0,95.639198,39.656084,86.96,Đường Hoàng Quốc Việt,Phường Nghĩa Đô,Quận Cầu Giấy,2020,...,False,False,False,True,False,False,False,False,False,False
1,4,3.0,37.0,94.627033,40.220172,116.22,Đường Kim Giang,Phường Kim Giang,Quận Thanh Xuân,2020,...,False,False,False,False,False,False,False,False,False,False
3,5,6.0,51.0,12.750000,4.000000,100.00,Đường Võng Thị,Phường Thụy Khuê,Quận Tây Hồ,2020,...,False,False,False,False,False,False,False,True,False,False
4,4,4.0,36.0,9.000000,4.000000,86.11,Đường Kim Giang,Phường Kim Giang,Quận Thanh Xuân,2020,...,False,False,False,False,False,False,False,False,False,False
5,7,10.0,46.0,12.100000,3.800000,104.35,Đường Yên Hòa,Phường Yên Hoà,Quận Cầu Giấy,2020,...,False,False,False,True,False,False,False,False,False,False


In [41]:
# Tính f từng category trong từng cột
freq_duong = df_impute['Đường'].value_counts(normalize=True)
freq_phuong = df_impute['Phường'].value_counts(normalize=True)

# Tạo cột mới với giá trị frequency encoding
df_impute['Đường_FE'] = df_impute['Đường'].map(freq_duong)
df_impute['Phường_FE'] = df_impute['Phường'].map(freq_phuong)

print(df_impute[['Đường', 'Đường_FE']].head())
print(df_impute[['Phường', 'Phường_FE']].head())


                   Đường  Đường_FE
0  Đường Hoàng Quốc Việt  0.007375
1        Đường Kim Giang  0.016214
3         Đường Võng Thị  0.002561
4        Đường Kim Giang  0.016214
5          Đường Yên Hòa  0.005634
             Phường  Phường_FE
0   Phường Nghĩa Đô   0.011707
1  Phường Kim Giang   0.010785
3  Phường Thụy Khuê   0.007434
4  Phường Kim Giang   0.010785
5    Phường Yên Hoà   0.017180


In [42]:
df_impute.columns

Index(['Số tầng', 'Số phòng ngủ', 'Diện tích', 'Dài', 'Rộng', 'Giá/m2',
       'Đường', 'Phường', 'Quận', 'Năm', 'Tháng', 'Ngày_',
       'Loại hình nhà ở_encode', 'Loai_hinh_nha_1', 'Loai_hinh_nha_2',
       'Loai_hinh_nha_3', 'Loai_hinh_nha_4', 'Giấy tờ pháp lý_encode',
       'Giay_to_1', 'Giay_to_2', 'Giay_to_3', 'Quận_encode', 'Quan__1',
       'Quan__2', 'Quan__3', 'Quan__4', 'Quan__5', 'Quan__6', 'Quan__7',
       'Quan__8', 'Quan__9', 'Quan__10', 'Quan__11', 'Quan__12', 'Đường_FE',
       'Phường_FE'],
      dtype='object')

In [43]:
df_impute.drop(columns=['Đường'], inplace=True)
df_impute.drop(columns=['Phường'], inplace=True)
df_impute.drop(columns=['Quận'], inplace=True)
df_impute.head()

,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2,Năm,Tháng,Ngày_,Loại hình nhà ở_encode,...,Quan__5,Quan__6,Quan__7,Quan__8,Quan__9,Quan__10,Quan__11,Quan__12,Đường_FE,Phường_FE
0,4,5.0,46.0,95.639198,39.656084,86.96,2020,8,5,1,...,False,True,False,False,False,False,False,False,0.007375,0.011707
1,4,3.0,37.0,94.627033,40.220172,116.22,2020,8,5,2,...,False,False,False,False,False,False,False,False,0.016214,0.010785
3,5,6.0,51.0,12.750000,4.000000,100.00,2020,8,5,1,...,False,False,False,False,False,True,False,False,0.002561,0.007434
4,4,4.0,36.0,9.000000,4.000000,86.11,2020,8,5,1,...,False,False,False,False,False,False,False,False,0.016214,0.010785
5,7,10.0,46.0,12.100000,3.800000,104.35,2020,8,5,1,...,False,True,False,False,False,False,False,False,0.005634,0.017180


In [44]:
df_impute.info()

<class 'pandas.core.frame.DataFrame'>
Index: 68335 entries, 0 to 82495
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Số tầng                 68335 non-null  int64  
 1   Số phòng ngủ            68335 non-null  float64
 2   Diện tích               68335 non-null  float64
 3   Dài                     68335 non-null  float64
 4   Rộng                    68335 non-null  float64
 5   Giá/m2                  68335 non-null  float64
 6   Năm                     68335 non-null  int32  
 7   Tháng                   68335 non-null  int32  
 8   Ngày_                   68335 non-null  int32  
 9   Loại hình nhà ở_encode  68335 non-null  int64  
 10  Loai_hinh_nha_1         68335 non-null  bool   
 11  Loai_hinh_nha_2         68335 non-null  bool   
 12  Loai_hinh_nha_3         68335 non-null  bool   
 13  Loai_hinh_nha_4         68335 non-null  bool   
 14  Giấy tờ pháp lý_encode  68335 non-null  int

In [45]:
df_impute.to_csv('VN_housing_dataset_processed.csv', index=False, encoding='utf-8')


In [46]:
nam = df_impute['Năm'].value_counts()
print(nam)

Năm
2020    68317
2019       18
Name: count, dtype: int64


In [47]:
from sklearn.preprocessing import RobustScaler


In [48]:
columns_to_scale = ["Số tầng", "Số phòng ngủ", "Diện tích", "Dài", "Rộng", "Giá/m2"]

scaler = RobustScaler()
df_impute[columns_to_scale] = scaler.fit_transform(df_impute[columns_to_scale])

df_impute.head()

,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2,Năm,Tháng,Ngày_,Loại hình nhà ở_encode,...,Quan__5,Quan__6,Quan__7,Quan__8,Quan__9,Quan__10,Quan__11,Quan__12,Đường_FE,Phường_FE
0,0.0,1.0,0.3750,0.018317,0.004051,-0.106757,2020,8,5,1,...,False,True,False,False,False,False,False,False,0.007375,0.011707
1,0.0,-1.0,-0.1875,0.006013,0.019829,0.684054,2020,8,5,2,...,False,False,False,False,False,False,False,False,0.016214,0.010785
3,1.0,2.0,0.6875,-0.989272,-0.993275,0.245676,2020,8,5,1,...,False,False,False,False,False,True,False,False,0.002561,0.007434
4,0.0,0.0,-0.2500,-1.034857,-0.993275,-0.129730,2020,8,5,1,...,False,False,False,False,False,False,False,False,0.016214,0.010785
5,3.0,6.0,0.3750,-0.997173,-0.998870,0.363243,2020,8,5,1,...,False,True,False,False,False,False,False,False,0.005634,0.017180


In [49]:
df_impute.drop(columns=['Năm', 'Tháng', 'Ngày_'], inplace=True)
df_impute.head()

,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2,Loại hình nhà ở_encode,Loai_hinh_nha_1,Loai_hinh_nha_2,Loai_hinh_nha_3,...,Quan__5,Quan__6,Quan__7,Quan__8,Quan__9,Quan__10,Quan__11,Quan__12,Đường_FE,Phường_FE
0,0.0,1.0,0.3750,0.018317,0.004051,-0.106757,1,True,False,False,...,False,True,False,False,False,False,False,False,0.007375,0.011707
1,0.0,-1.0,-0.1875,0.006013,0.019829,0.684054,2,False,True,False,...,False,False,False,False,False,False,False,False,0.016214,0.010785
3,1.0,2.0,0.6875,-0.989272,-0.993275,0.245676,1,True,False,False,...,False,False,False,False,False,True,False,False,0.002561,0.007434
4,0.0,0.0,-0.2500,-1.034857,-0.993275,-0.129730,1,True,False,False,...,False,False,False,False,False,False,False,False,0.016214,0.010785
5,3.0,6.0,0.3750,-0.997173,-0.998870,0.363243,1,True,False,False,...,False,True,False,False,False,False,False,False,0.005634,0.017180


**FEATURE SELECTION**

In [50]:
df_impute.info()

<class 'pandas.core.frame.DataFrame'>
Index: 68335 entries, 0 to 82495
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Số tầng                 68335 non-null  float64
 1   Số phòng ngủ            68335 non-null  float64
 2   Diện tích               68335 non-null  float64
 3   Dài                     68335 non-null  float64
 4   Rộng                    68335 non-null  float64
 5   Giá/m2                  68335 non-null  float64
 6   Loại hình nhà ở_encode  68335 non-null  int64  
 7   Loai_hinh_nha_1         68335 non-null  bool   
 8   Loai_hinh_nha_2         68335 non-null  bool   
 9   Loai_hinh_nha_3         68335 non-null  bool   
 10  Loai_hinh_nha_4         68335 non-null  bool   
 11  Giấy tờ pháp lý_encode  68335 non-null  int64  
 12  Giay_to_1               68335 non-null  bool   
 13  Giay_to_2               68335 non-null  bool   
 14  Giay_to_3               68335 non-null  boo

In [51]:
df_impute_ = df_impute.copy()

**BỎ 1 CỘT TRONG MỖI NHÓM ONE-HOT giảm redundancy, tăng hiệu quả feature selection**

In [52]:
df_impute_.drop(columns=['Loai_hinh_nha_2','Giay_to_3','Quan__4'], inplace=True)
df_impute_.info()

<class 'pandas.core.frame.DataFrame'>
Index: 68335 entries, 0 to 82495
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Số tầng                 68335 non-null  float64
 1   Số phòng ngủ            68335 non-null  float64
 2   Diện tích               68335 non-null  float64
 3   Dài                     68335 non-null  float64
 4   Rộng                    68335 non-null  float64
 5   Giá/m2                  68335 non-null  float64
 6   Loại hình nhà ở_encode  68335 non-null  int64  
 7   Loai_hinh_nha_1         68335 non-null  bool   
 8   Loai_hinh_nha_3         68335 non-null  bool   
 9   Loai_hinh_nha_4         68335 non-null  bool   
 10  Giấy tờ pháp lý_encode  68335 non-null  int64  
 11  Giay_to_1               68335 non-null  bool   
 12  Giay_to_2               68335 non-null  bool   
 13  Quận_encode             68335 non-null  int64  
 14  Quan__1                 68335 non-null  boo

In [53]:
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample

**Minimum Redundancy Maximum Relevance (mRMR) selection**

In [54]:
y = df_impute_['Giá/m2']
X = df_impute_.drop(columns=['Giá/m2'])

In [55]:
X_enc = X.copy()
for col in X_enc.select_dtypes(include=['bool', 'object', 'int64']).columns:
  le = LabelEncoder()
  X_enc[col] = le.fit_transform(X_enc[col].astype(str))

In [56]:
X_s, y_s = resample(X_enc, y, n_samples=5000, random_state=42)

discrete_mask = X_s.dtypes != 'float64'
mi_rel = mutual_info_regression(X_s, y_s, discrete_features=discrete_mask, random_state=42)
mi_rel = pd.Series(mi_rel, index=X_enc.columns)

In [57]:
features = X_enc.columns.tolist()
mi_between = pd.DataFrame(0.0, index=features, columns=features)
for f1 in features:
  for f2 in features:
    if f1 != f2:
      mi_between.loc[f1, f2] = mutual_info_regression(
        X_s[[f1]], X_s[f2],
        discrete_features=[discrete_mask[f1]],
        random_state=42
      )[0]

In [58]:
selected = []
scores = []
for k in range(10):  # select top 10 features
  if not selected:
    # pick feature with max relevance
    best = mi_rel.idxmax()
  else:
    # maximize: I(f;Y) - (1/|S|) * Σ_{s∈S} I(f;s)
    cand_scores = {}
    for f in features:
      if f not in selected:
        red = mi_between.loc[f, selected].mean()
        cand_scores[f] = mi_rel[f] - red
    best = max(cand_scores, key=cand_scores.get)
  selected.append(best)
  avg_red = mi_between.loc[best, selected[:-1]].mean() if len(selected)>1 else 0.0
  scores.append((best, mi_rel[best], avg_red, mi_rel[best] - avg_red))


In [61]:
selected_df = pd.DataFrame(scores, columns=['feature','relevance','redundancy','mRMR_score'])
selected_df

,feature,relevance,redundancy,mRMR_score
0,Dài,1.340143,0.000000,1.340143
1,Diện tích,0.861250,0.284577,0.576673
2,Quận_encode,0.190472,0.100439,0.090033
3,Rộng,0.962457,0.687916,0.274541
4,Loai_hinh_nha_1,0.089852,0.042772,0.047079
5,Số tầng,0.125435,0.075675,0.049760
6,Giay_to_1,0.000541,0.000830,-0.000290
7,Quan__12,0.005801,0.006434,-0.000633
8,Số phòng ngủ,0.101262,0.097887,0.003375
9,Loại hình nhà ở_encode,0.095450,0.088853,0.006597


In [62]:
#Relevance (Độ liên quan): Mức độ thông tin của đặc trưng đối với biến mục tiêu (Giá/m2). Giá trị càng cao càng tốt.

#Redundancy (Độ lặp lại): Mức độ trùng lặp thông tin của đặc trưng với các đặc trưng đã chọn. Giá trị càng thấp càng tốt.

#mRMR_score: Hiệu số giữa Relevance và Redundancy, đại diện cho độ “ưu tiên” chọn đặc trưng.

In [77]:
df_impute_.info()

<class 'pandas.core.frame.DataFrame'>
Index: 68335 entries, 0 to 82495
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Số tầng                 68335 non-null  float64
 1   Số phòng ngủ            68335 non-null  float64
 2   Diện tích               68335 non-null  float64
 3   Dài                     68335 non-null  float64
 4   Rộng                    68335 non-null  float64
 5   Giá/m2                  68335 non-null  float64
 6   Loại hình nhà ở_encode  68335 non-null  int64  
 7   Loai_hinh_nha_1         68335 non-null  bool   
 8   Loai_hinh_nha_3         68335 non-null  bool   
 9   Loai_hinh_nha_4         68335 non-null  bool   
 10  Giấy tờ pháp lý_encode  68335 non-null  int64  
 11  Giay_to_1               68335 non-null  bool   
 12  Giay_to_2               68335 non-null  bool   
 13  Quận_encode             68335 non-null  int64  
 14  Quan__1                 68335 non-null  boo

In [78]:
features_ = ["Dài", "Rộng", "Diện tích", "Quận_encode", "Loại hình nhà ở_encode", "Số tầng", "Giá/m2"]

df_selected_features = df_impute_[features_]

In [79]:
df_selected_features

,Dài,Rộng,Diện tích,Quận_encode,Loại hình nhà ở_encode,Số tầng,Giá/m2
0,0.018317,0.004051,0.3750,6,1,0.0,-0.106757
1,0.006013,0.019829,-0.1875,2,2,0.0,0.684054
3,-0.989272,-0.993275,0.6875,10,1,1.0,0.245676
4,-1.034857,-0.993275,-0.2500,2,1,0.0,-0.129730
5,-0.997173,-0.998870,0.3750,6,1,3.0,0.363243
...,...,...,...,...,...,...,...
82491,0.029093,0.005867,-0.1250,11,3,0.0,-0.252162
82492,-0.113308,0.088354,0.6250,6,2,1.0,5.434865
82493,-0.145593,0.110201,0.0625,6,2,1.0,6.771622
82494,0.012699,0.012006,1.2500,6,1,1.0,0.290811


In [80]:
df_selected_features.to_csv('df_selected_features.csv', index=False, encoding='utf-8')


**Tăng cường dữ liệu bằng kỹ thuật jittering (thêm nhiễu Gaussian nhỏ) trên các biến số liên tục, giữ nguyên các biến phân loại**

In [81]:
def jitter_augmentation(df, numeric_cols, n_new_samples=10000, jitter_scale=0.01, random_state=42):
  np.random.seed(random_state)

  # Lấy mẫu ngẫu nhiên có hoàn lại
  sampled_df = df.sample(n=n_new_samples, replace=True).reset_index(drop=True)

  # Thêm nhiễu Gaussian nhỏ vào các cột số liên tục
  for col in numeric_cols:
    std = df[col].std()
    noise = np.random.normal(0, std * jitter_scale, n_new_samples)
    sampled_df[col] = sampled_df[col] + noise

  # Kết hợp dữ liệu gốc và dữ liệu tăng cường
  augmented_df = pd.concat([df, sampled_df], ignore_index=True)
  return augmented_df

In [82]:
numeric_features = ['Dài', 'Rộng', 'Diện tích', 'Số tầng']

df_aug = jitter_augmentation(df_selected_features, numeric_features, n_new_samples=10000, jitter_scale=0.01)

In [84]:
print(f"Original samples: {len(df_selected_features)}, After augmentation: {len(df_aug)}")

Original samples: 68335, After augmentation: 78335


In [85]:
target = 'Giá/m2'

In [86]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [88]:
X_ = df_aug.drop(columns=[target])
y_ = df_aug[target]

X_train_val, X_test, y_train_val, y_test = train_test_split(
    X_, y_, test_size=0.2, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.25, random_state=42
)


In [95]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Khởi tạo mô hình Random Forest Regression với 100 cây
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Huấn luyện trên tập train
model.fit(X_train, y_train)

# Dự đoán trên tập validation
y_val_pred = model.predict(X_val)
val_mse = mean_squared_error(y_val, y_val_pred)
val_r2 = r2_score(y_val, y_val_pred)

print(f"Validation MSE: {val_mse:.4f}")
print(f"Validation R2: {val_r2:.4f}")

# Dự đoán trên tập test
y_test_pred = model.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"Test MSE: {test_mse:.4f}")
print(f"Test R2: {test_r2:.4f}")


Validation MSE: 1.0943
Validation R2: 0.6310
Test MSE: 1.0990
Test R2: 0.6487
